# This data was taken from NAHNES survey carried in USA by CDC 


In [1]:
import pandas as pd
import os
from functools import reduce

def process_nhanes_year(directory_path):
    print(f"--- ⏳ Processing directory: {directory_path} ---")
    dataframes = []
    for file in os.listdir(directory_path):
        if file.endswith(".xpt"):
            filepath = os.path.join(directory_path, file)
            try:
                df = pd.read_sas(filepath)
                dataframes.append(df)
            except Exception as e:
                print(f"  [!] Warning: Could not read file {filepath}. Error: {e}")
    
    if not dataframes:
        print(f"  [!] No .xpt files found in {directory_path}")
        return pd.DataFrame() 
    combined_df = reduce(
        lambda left, right: pd.merge(left, right, on='SEQN', how='outer', suffixes=('_left', '_right')),
        dataframes
    )
    combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]
    print(f"  ✅ Successfully merged {len(dataframes)} files. Final shape: {combined_df.shape}")
    return combined_df

In [2]:
combined_df_2023 = process_nhanes_year(r"C:\Users\HP\Downloads\2023\\")
combined_df_2017 = process_nhanes_year(r"C:\Users\HP\Downloads\2017\\")
combined_df_2015 = process_nhanes_year(r"C:\Users\HP\Downloads\2015\\")
combined_df_2013 = process_nhanes_year(r"C:\Users\HP\Downloads\2013\\")
combined_df_2011 = process_nhanes_year(r"C:\Users\HP\Downloads\2011\\")

--- ⏳ Processing directory: C:\Users\HP\Downloads\2023\\ ---
  ✅ Successfully merged 7 files. Final shape: (11933, 85)
--- ⏳ Processing directory: C:\Users\HP\Downloads\2017\\ ---
  ✅ Successfully merged 7 files. Final shape: (15560, 113)
--- ⏳ Processing directory: C:\Users\HP\Downloads\2015\\ ---
  ✅ Successfully merged 7 files. Final shape: (9971, 196)
--- ⏳ Processing directory: C:\Users\HP\Downloads\2013\\ ---
  ✅ Successfully merged 7 files. Final shape: (10175, 188)
--- ⏳ Processing directory: C:\Users\HP\Downloads\2011\\ ---
  ✅ Successfully merged 7 files. Final shape: (9756, 190)


In [3]:
column_name_mappings = {
        "SEQN": "Respondent Sequence Number (ID)",
        "RIAGENDR": "Gender of the participant (1=Male, 2=Female)",
        "RIDAGEYR": "Age in years (Top-coded at 80)",
        "RIDRETH3": "Race/Hispanic origin category",
        "DMDEDUC2": "Highest level of education attained",
        "DMDMARTZ": "Marital status",
        "INDFMPIR": "Ratio of family income to poverty",
        "WTINT2YR": "Interview sample weight (2-year)",
        "WTMEC2YR": "MEC sample weight (2-year)",
        "BMXWT": "Weight (kg)",
        "BMXHT": "Standing Height (cm)",
        "BMXBMI": "Body Mass Index (BMI: kg/m^2)",
        "BMXWAIST": "Waist Circumference (cm)",
        "BMXHIP": "Hip Circumference (cm)",
        "BMXARMC": "Mid-Upper Arm Circumference (cm)",
        "BPXOSY1": "Systolic Blood Pressure, 1st reading (mmHg)",
        "BPXOSY3": "Systolic Blood Pressure, 3rd reading (mmHg)",
        "BPXSY1": "Systolic Blood Pressure, 1st reading (mmHg)",
        "BPXDI1": "Diastolic Blood Pressure, 1st reading (mmHg)",
        "BPXODI1": "Diastolic Blood Pressure, 1st reading (mmHg)",
        "BPXODI3": "Diastolic Blood Pressure, 3rd reading (mmHg)",
        "BPXOPLS1": "Pulse rate (beats per minute), 1st reading",
        "DIQ010": "Ever told by doctor they had Diabetes?",
        "DID040": "Age when first diagnosed with diabetes",
        "DIQ160": "Ever told by doctor they had Prediabetes?",
        "DIQ050": "Currently taking Insulin?",
        "DIQ070": "Currently taking diabetic Pills to lower blood sugar?",
        "DIQ180": "Had a blood test for high blood sugar in the past three years?",
        "SEQN": "Respondent Sequence Number (ID)",
        "LBXGH": "Glycohemoglobin (HbA1c) value (%)",
        "WTPH2YR": "Subsample weight for laboratory data",
        "SMQ020": "Ever smoked at least 100 cigarettes in life?",
        "SMQ040": "Currently Smoke? (Every day, some days, or not at all)",
        "SMD650": "Number of cigarettes smoked per time unit",
        "SMD641": "Time since last smoked regularly",
        "ALQ111": "Drank alcohol in the past 12 months?",
        "ALQ121": "Frequency of drinking alcohol in the past 12 months",
        "ALQ130": "Average number of drinks per day when drinking",
        "ALQ142": "Maximum number of drinks consumed in one day in the past 12 months",
        "DMDMARTL": "Marital status",
        "ALQ110": "Drank alcohol in the past 12 months?"
}

In [4]:
# Select the most important 20 columns for Health Risk Analysis

selected_columns = [
    'Respondent Sequence Number (ID)',
    # 🧍 Demographics
    'Gender of the participant (1=Male, 2=Female)',
    'Age in years (Top-coded at 80)',
    'Race/Hispanic origin category',
    'Highest level of education attained',
    'Ratio of family income to poverty',

    # ⚖️ Body Measures
    'Weight (kg)',
    'Standing Height (cm)',
    'Body Mass Index (BMI: kg/m^2)',

    # ❤️ Blood Pressure
    'Systolic Blood Pressure, 1st reading (mmHg)',
    'Diastolic Blood Pressure, 1st reading (mmHg)',

    # 🍬 Diabetes
    'Ever told by doctor they had Diabetes?',
    'Age when first diagnosed with diabetes',
    'Currently taking diabetic Pills to lower blood sugar?',

    # 🩸 Lab Data
    'Glycohemoglobin (HbA1c) value (%)',

    # 🚬 Smoking
    'Ever smoked at least 100 cigarettes in life?',
    'Currently Smoke? (Every day, some days, or not at all)',

    # 🍺 Alcohol
    'Drank alcohol in the past 12 months?',
    'Average number of drinks per day when drinking',

    # 🧘 Lifestyle
    'Marital status'

]



In [5]:
raw_dataframes_list = [
    combined_df_2023, 
    combined_df_2017, 
    combined_df_2015, 
    combined_df_2013, 
    combined_df_2011
]
all_filtered_dfs = [] 
for df in raw_dataframes_list:
    decoded_df = df.rename(columns=column_name_mappings)
    existing_cols_in_df = [col for col in selected_columns if col in decoded_df.columns]
    filtered_df = decoded_df[existing_cols_in_df].copy()
    all_filtered_dfs.append(filtered_df)
print(f"--- ✅ Done processing all {len(all_filtered_dfs)} dataframes! ---")
final_combined_df = pd.concat(all_filtered_dfs, ignore_index=True)
print(f"Shape of the final combined dataframe: {final_combined_df.shape}")
print(f"Columns of the final combined dataframe: {final_combined_df.columns}")

--- ✅ Done processing all 5 dataframes! ---
Shape of the final combined dataframe: (57395, 20)
Columns of the final combined dataframe: Index(['Respondent Sequence Number (ID)',
       'Gender of the participant (1=Male, 2=Female)',
       'Age in years (Top-coded at 80)', 'Race/Hispanic origin category',
       'Highest level of education attained',
       'Ratio of family income to poverty', 'Weight (kg)',
       'Standing Height (cm)', 'Body Mass Index (BMI: kg/m^2)',
       'Systolic Blood Pressure, 1st reading (mmHg)',
       'Diastolic Blood Pressure, 1st reading (mmHg)',
       'Ever told by doctor they had Diabetes?',
       'Age when first diagnosed with diabetes',
       'Currently taking diabetic Pills to lower blood sugar?',
       'Glycohemoglobin (HbA1c) value (%)',
       'Ever smoked at least 100 cigarettes in life?',
       'Currently Smoke? (Every day, some days, or not at all)',
       'Drank alcohol in the past 12 months?',
       'Average number of drinks per day w

In [6]:
final_combined_df.to_csv(r"C:\Users\HP\Desktop\NAHNES.csv", index=False)